In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sqlalchemy import create_engine
import pickle
from sklearn.decomposition import PCA
import plotly.express as px

DB_PATH = "C:/Users/Khalid/Desktop/PFB/sp500.db"
engine = create_engine(f'sqlite:///{DB_PATH}')


query_empresas = "SELECT * FROM empresas_sp500"
empresas_df = pd.read_sql(query_empresas, engine)

id_empresa_col = 'id_empresa'
simbolo_col = 'Symbol'
sector_col = 'GICS Sector'
industria_col = 'GICS Sub-Industry'


query_precios = """
SELECT * FROM precios_historicos
"""  # Ajusta el nombre si cambió
precios_df = pd.read_sql(query_precios, engine)


if 'fecha' in precios_df.columns:
    precios_df['fecha'] = pd.to_datetime(precios_df['fecha'])


resumen_precios = precios_df.groupby(id_empresa_col).agg({
    'precio_apertura': ['mean', 'std'],
    'precio_cierre': ['mean', 'std'],
    'maximo': ['mean', 'std'],
    'minimo': ['mean', 'std'],
    'volumen': ['mean', 'std']
}).reset_index()

resumen_precios.columns = [
    id_empresa_col if col[0] == id_empresa_col else '_'.join(col).rstrip('_')
    for col in resumen_precios.columns]


caracteristicas_empresas = empresas_df[[id_empresa_col, simbolo_col, sector_col, industria_col]]


caracteristicas_combinadas = pd.merge(
    caracteristicas_empresas,
    resumen_precios,
    on=id_empresa_col,
    how='inner')


caracteristicas_combinadas = pd.get_dummies(
    caracteristicas_combinadas, columns=[sector_col, industria_col], drop_first=True)


caracteristicas_finales = caracteristicas_combinadas.drop(columns=[id_empresa_col, simbolo_col])
scaler = StandardScaler()
caracteristicas_normalizadas = scaler.fit_transform(caracteristicas_finales)

num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
caracteristicas_combinadas['cluster'] = kmeans.fit_predict(caracteristicas_normalizadas)

# Entrenar modelo supervisado
X = caracteristicas_finales
y = caracteristicas_combinadas['cluster']

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo de clasificación
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = rf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


caracteristicas_combinadas[['id_empresa', 'cluster']].to_sql(
    'empresas_clusters',
    con=engine,
    if_exists='replace',
    index=False)

with open('modelo_entrenamiento.pkl', 'wb') as f:
    pickle.dump(rf, f)

print("Modelo entrenado y guardado como 'modelo_entrenamiento.pkl'")

# 8. Visualizar clústeres
pca = PCA(n_components=2)
pca_result = pca.fit_transform(caracteristicas_normalizadas)

# Crear un DataFrame para los resultados de PCA
pca_df = pd.DataFrame({
    'Componente Principal 1': pca_result[:, 0],
    'Componente Principal 2': pca_result[:, 1],
    'Clúster': caracteristicas_combinadas['cluster']})

fig = px.scatter(
    pca_df,
    x='Componente Principal 1',
    y='Componente Principal 2',
    color='Clúster',
    title='Visualización de Clústeres (PCA)',
    color_continuous_scale=px.colors.sequential.Viridis,
    labels={'Clúster': 'Clúster'},
    template='plotly_white')
fig.show()


C:\Users\Khalid\AppData\Local\Temp\ipykernel_35432\1748246448.py:32: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  precios_df['fecha'] = pd.to_datetime(precios_df['fecha'])


Accuracy: 0.9702970297029703
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98        98
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         1

    accuracy                           0.97       101
   macro avg       0.32      0.33      0.33       101
weighted avg       0.94      0.97      0.96       101

Modelo entrenado y guardado como 'modelo_entrenamiento.pkl'


c:\Users\Khalid\Desktop\PFB\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Khalid\Desktop\PFB\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Khalid\Desktop\PFB\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
